In [8]:
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
import torchvision
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sym import LCN, ProgressBar
device = "cuda:0" if torch.cuda.is_available() else "cpu"
plt.style.use("dark_paper")

In [2]:
# torch.save(testset,"./data/testset.pt")
# torch.save(trainset,"./data/trainset.pt")
testset = torch.load("./data/testset.pt")
trainset = torch.load("./data/trainset.pt")

In [4]:
class Model(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.Loss = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        yhat = self(x)
        return self.Loss(yhat, y)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        yhat = self(x)
        loss = self.Loss(yhat, y)
        preds = torch.argmax(yhat, dim=1)
        acc = accuracy(preds, y)
        # Calling self.log will surface up scalars for you in TensorBoard
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self, learning_rate=1e-2):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        return optimizer

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            self.mnist_train, self.mnist_val = torch.utils.data.random_split(trainset, [
                                                                             55000, 5000])
        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.mnist_test = testset

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_train, num_workers=6, batch_size=320)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_val, num_workers=6, batch_size=320)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_test, num_workers=6, batch_size=320)

In [5]:
class SFCN(torch.nn.Module):
    def __init__(self,input_dim=280,nfilters=32,pool=100,C=10):
        super().__init__()
        self.nfilters = nfilters
        self.pool_kernel = pool
        self.fc1 = torch.nn.Linear(input_dim**2, nfilters*pool)
        self.pool = torch.nn.MaxPool1d(pool,1)
        self.out = torch.nn.Linear(nfilters, C)
    def forward(self,x):
        x = x.flatten(start_dim=1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.pool(x.view((-1,self.nfilters,self.pool_kernel)))
        return self.out(x.view(-1,self.nfilters))

In [7]:
bar = ProgressBar()
trainer = pl.Trainer(gpus=1, max_epochs=10, progress_bar_refresh_rate=20,callbacks=[bar])
sfcn = Model(SFCN())
# Train the model
trainer.fit(sfcn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SFCN             | 250 M 
1 | Loss  | CrossEntropyLoss | 0     


Epoch 0:   0%|          | 3/3016 [00:00<02:34, 19.53it/s, loss=43.317, v_num=17]

/home/kitouni/anaconda3/envs/gnn/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0: 100%|##########| 3016/3016 [02:27<00:00, 20.45it/s, loss=286.556, v_num=17, val_loss=148, val_acc=0.848]


1

In [19]:
nfilters = 32
cnn = Model(torch.nn.Sequential(torch.nn.Conv2d(1, nfilters, kernel_size=28, stride=28),
                          #torch.nn.BatchNorm2d(nfilters),
                          torch.nn.ReLU(),
                          torch.nn.MaxPool2d(10, 1),
                          torch.nn.Flatten(),
                          torch.nn.Linear(nfilters, 10)))  # (10)**2
bar = ProgressBar()
trainer = pl.Trainer(gpus=1, max_epochs=10, progress_bar_refresh_rate=1,callbacks=[bar])
trainer.fit(cnn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | Sequential       | 25.4 K
1 | Loss  | CrossEntropyLoss | 0     


Epoch 9: 100%|##########| 188/188 [00:08<00:00, 21.16it/s, loss=0.448, v_num=22, val_loss=0.491, val_acc=0.889]


1

In [17]:
lcn = Model(LCN(f=32))